In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# add . to module name
import sys
sys.path.append('./src/')
from package.footpaths import *

In [3]:
from package.logger import setup
setup("INFO")

NameError: name 'edges' is not defined

In [8]:
from package import storage,build, raptor, strtime,key
from package.gtfs import archive

In [9]:
footpaths_dict = storage.read_any_dict("./data/footpaths.pkl")
footpaths_dict = footpaths_dict["footpaths"]

structs_dict = storage.read_any_dict("./data/structs.pkl")
build.validate_structs_dict(structs_dict)


In [10]:
(
	stop_times_by_trip,
	trip_ids_by_route,
	stops_by_route,
	idx_by_stop_by_route,
	routes_by_stop,
	times_by_stop_by_trip,
	stop_id_set,
	route_id_set,
	trip_id_set,
) = raptor.unpack_structs(structs_dict)

In [15]:
times_by_stop_by_trip

{'1-12-001-57.1.11:031600-27-2_CBCBA11D-C155-45D3-8ED4-B02D0133C4AB': {'30': (12720,
   12720),
  '32': (12780, 12780),
  '36': (12840, 12840),
  '35': (12960, 13080),
  '305': (13200, 13200),
  '304': (13260, 13260),
  '303': (13380, 13380),
  '336': (13440, 13440)},
 '1-121-001-407.2.21:045300-41-256_0EC7C2E7-BB2C-49F9-82C5-AFF1013D6D54': {'334': (20880,
   20880),
  '900': (20940, 20940),
  '332': (21060, 21060),
  '773': (21120, 21120),
  '789': (21180, 21180),
  '325': (21240, 21240),
  '310': (21480, 21480),
  '838': (21660, 21660),
  '303': (21720, 21720)},
 '1-124-001-9.2.28:051100-12-256_F5F47D52-2147-4947-87A1-AFF1013D68A3': {'9': (18660,
   18660),
  '317': (19140, 19140)},
 '1-127-001-35.2.22:043100-35-257_D7BC3B30-E11A-4F52-B402-B02D012A79C3': {'35': (16260,
   16260),
  '36': (16320, 16320),
  '38': (16440, 16440),
  '312': (16560, 16560),
  '837': (16620, 16620),
  '311': (16680, 16680),
  '750': (16740, 16740),
  '815': (16860, 16860),
  '327': (16980, 16980),
  '328': 

In [66]:
first_trip_id = "96-2-001-343.1.12:151400-15-257_5BA79AD0-5693-49CD-87F9-B020013BDC8C"
arrival_time, departure_time = times_by_stop_by_trip[first_trip_id]["35"]
print(arrival_time, strtime.seconds_to_str_time(arrival_time))

55320 15:22:00


In [67]:
routes_by_stop["35"]
route12 = [
 '100012_0_A',
 '100012_0_B',
 '100012_0_D',
 '100012_1_A', # right direction - wrong route
 '100012_1_B', # right direction - right route
]
route_id = route12[4]


In [68]:
idx_by_stop_by_route["100012_1_B"]["52"]
# -> correct route 

17

In [69]:
def get_departure_time(trip_id, stop_id: str) -> int:
	assert stop_id is not None

	if trip_id is None:
		return sys.maxsize

	departure_time = times_by_stop_by_trip[trip_id][stop_id][1]
	assert type(departure_time) == int
	return departure_time

def get_arrival_time(trip_id: str, stop_id: str) -> int:
	assert trip_id is not None
	assert stop_id is not None

	arrival_time = times_by_stop_by_trip[trip_id][stop_id][0]
	assert type(arrival_time) == int
	return arrival_time

def earliest_trip(
	route_id: str, stop_id: str, arrival_time: int, change_time: int
):
	trip_ids = trip_ids_by_route[route_id]
	for trip_id in trip_ids:
		departure_time = get_departure_time(trip_id, stop_id)
		if departure_time >= arrival_time + change_time:
			return trip_id, departure_time


In [70]:
t = "251-16-001-124.1.11:143000-23-2_8D980DEA-2A41-4FF7-937B-AF5F0118D688"
strtime.seconds_to_str_time(get_arrival_time(t, "106"))

'14:34:00'

In [71]:
trip, departure_time = earliest_trip(route_id, "35", arrival_time, 180)
print(trip, departure_time, strtime.seconds_to_str_time(departure_time))

605-12-001-417.1.12:150600-27-254_A71C2242-7C2C-417E-B01B-AFB20109C38B 55800 15:30:00


In [72]:
import pandas as pd
import geopandas as gpd

from package.key import (
    STOP_TIMES_KEY,
    TRIPS_KEY,
    STOPS_KEY,
)
from package.logger import Timed
from package.gtfs import archive,clean


dfs = archive.read_dfs("./data/cologne_gtfs.zip")
trips_df, stop_times_df, stops_df = (
	dfs[TRIPS_KEY],
	dfs[STOP_TIMES_KEY],
	dfs[STOPS_KEY],
)

clean.split_routes_by_direction(trips_df)
paths_df = clean.create_paths_df(trips_df, stop_times_df)
paths_df = clean.add_unique_route_ids(paths_df)
paths_df






,route_id,trip_id,path,new_route_id
0,100001_0,1-1-001-28.1.11:030100-15-257_14F0FAC7-4465-49...,"['28', '27', '2', '1', '39', '802', '44', '512']",100001_0_A
1,100001_0,10-1-001-702.1.11:101800-37-2_95C318A3-D7D3-43...,"['180', '179', '178', '144', '143', '28', '27'...",100001_0_B
2,100001_0,100-1-001-702.1.11:120800-37-2_80C40FD7-524C-4...,"['180', '179', '178', '144', '143', '28', '27'...",100001_0_B
3,100001_0,100-1-001-702.1.11:122800-37-255_D8AA4985-BA32...,"['180', '179', '178', '144', '143', '28', '27'...",100001_0_B
4,100001_0,101-1-001-702.1.11:123500-37-2_1A3BE569-B3DD-4...,"['180', '179', '178', '144', '143', '28', '27'...",100001_0_B
...,...,...,...,...
58029,300025_1,725050-25-003-3452.2.21:181000-51-63_9004369F-...,"['640', '962', '961', '959', '806', '51', '257...",300025_1_A
58030,300025_1,725054-25-003-3432.2.21:180500-34-63_882A3FAC-...,"['640', '962', '961', '959', '806', '51', '257...",300025_1_A
58031,300025_1,725054-25-003-3452.2.21:191000-51-63_6792C5F8-...,"['640', '962', '961', '959', '806', '51', '257...",300025_1_A
58032,300025_1,725058-25-003-3432.2.21:190500-34-63_8FDC73C8-...,"['640', '962', '961', '959', '806', '51', '257...",300025_1_A


In [73]:
problem_route =  "100016_0_A"
problem_path = paths_df[paths_df["new_route_id"] == problem_route].iloc[0]["path"]
print(problem_path)
problem_path = ['106', '897', '76', '65', '17', '18', '19', '22', '23', '3', '2', '7', '752', '9', '35', '34', '316', '317', '818', '343']
print(len(problem_path))
print(problem_path)


['106', '897', '76', '65', '17', '18', '19', '22', '23', '3', '2', '7', '752', '9', '35', '34', '316', '317', '818', '343']
20
['106', '897', '76', '65', '17', '18', '19', '22', '23', '3', '2', '7', '752', '9', '35', '34', '316', '317', '818', '343']


In [74]:
stop_seq_tuple_list = stops_by_route[problem_route]
stop_seq_tuple_list.sort(key=lambda x: x[1])
stop_seq_tuple_list

[('106', 7),
 ('897', 8),
 ('76', 9),
 ('65', 10),
 ('17', 11),
 ('18', 12),
 ('19', 13),
 ('22', 14),
 ('23', 15),
 ('3', 16),
 ('2', 17),
 ('7', 18),
 ('752', 19),
 ('9', 20),
 ('35', 21),
 ('34', 22),
 ('316', 23),
 ('317', 24),
 ('818', 25),
 ('343', 26)]